<a href="https://colab.research.google.com/github/Hanyan-Wu/MachineLearningLab/blob/main/Credit_Fraud_Detecting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hanyan Wu, UID: 406069524

# 1.) Import the Credit Card Fraud Data From CCLE

In [1]:
import pandas as pd
from google.colab import drive
import matplotlib.pyplot as plt
import numpy as np

In [2]:
drive.mount('/content/gdrive/', force_remount = True)

Mounted at /content/gdrive/


In [3]:
df = pd.read_csv("/content/gdrive/MyDrive/ColabNotebooks/fraudTest.csv")

In [4]:
df.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,...,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,...,40.3207,-110.4360,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0
2,2,2020-06-21 12:14:53,3598215285024754,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,...,40.6729,-73.5365,34496,"Librarian, public",1970-10-21,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0
3,3,2020-06-21 12:15:15,3591919803438423,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,...,28.5697,-80.8191,54767,Set designer,1987-07-25,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0
4,4,2020-06-21 12:15:17,3526826139003047,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,...,44.2529,-85.0170,1126,Furniture designer,1955-07-06,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0


# 2.) Select four columns to use as features (one just be trans_date_trans)

In [5]:
df_select = df[['trans_date_trans_time', 'category', 'amt', 'city_pop']]
df_select.columns

Index(['trans_date_trans_time', 'category', 'amt', 'city_pop'], dtype='object')

# 3.) Create a unique variable out of trans_date.

In [6]:
df_select['trans_date_trans_time'] = pd.to_datetime(df_select['trans_date_trans_time'])

<ipython-input-6-035beec6fe23>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select['trans_date_trans_time'] = pd.to_datetime(df_select['trans_date_trans_time'])


In [7]:
df_select['trans_date_trans_time']

0        2020-06-21 12:14:25
1        2020-06-21 12:14:33
2        2020-06-21 12:14:53
3        2020-06-21 12:15:15
4        2020-06-21 12:15:17
                 ...        
555714   2020-12-31 23:59:07
555715   2020-12-31 23:59:09
555716   2020-12-31 23:59:15
555717   2020-12-31 23:59:24
555718   2020-12-31 23:59:34
Name: trans_date_trans_time, Length: 555719, dtype: datetime64[ns]

the higher the second

In [8]:
# frauds usually happen between 1 a.m. and 6 a.m.
df_select['time_var'] = [i.hour for i in df_select['trans_date_trans_time']]
df_select['time_var'] = [1 if 7 > i > 0 else 0 for i in df_select['time_var']]
df_select['time_var'].describe()

<ipython-input-8-f536a530fcb4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select['time_var'] = [i.hour for i in df_select['trans_date_trans_time']]
<ipython-input-8-f536a530fcb4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select['time_var'] = [1 if 7 > i > 0 else 0 for i in df_select['time_var']]


count    555719.000000
mean          0.195959
std           0.396937
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: time_var, dtype: float64

In [9]:
X = pd.get_dummies(df_select, ['category'], drop_first= True).drop(['trans_date_trans_time'], axis= 1)
y = df['is_fraud']
X.head()

,amt,city_pop,time_var,category_food_dining,category_gas_transport,category_grocery_net,category_grocery_pos,category_health_fitness,category_home,category_kids_pets,category_misc_net,category_misc_pos,category_personal_care,category_shopping_net,category_shopping_pos,category_travel
0,2.86,333497,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,29.84,302,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,41.28,34496,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,60.05,54767,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,3.19,1126,0,0,0,0,0,0,0,0,0,0,0,0,0,1


# 4.) Oversample the data (this will be your training data).

In [10]:
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from collections import Counter

In [11]:
## Split the data into training and test sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Instantiate the oversampler
ros = RandomOverSampler(random_state=42)

# Fit and transform the training data
resample_X, resample_y = ros.fit_resample(X, y)

# Check the number of records after over sampling
print(sorted(Counter(resample_y).items()))

[(0, 553574), (1, 553574)]


# 5.) Train a Logistic regression.

In [12]:
from sklearn.linear_model import LogisticRegression

In [13]:
log_reg = LogisticRegression().fit(resample_X, resample_y)

# 6.) The company you are working for wants to target at a False Positive rate of 5% what threshold should you use? (Use oversampled data)

In [14]:
from sklearn.metrics import roc_curve

In [15]:
# Predict the probability of the positive class for each sample
y_proba = log_reg.predict_proba(resample_X)[:, 1]

# Compute the false positive rate and true positive rate
fpr, tpr, thresholds = roc_curve(resample_y, y_proba)

# Find the threshold closest to a false positive rate of 5%
idx = np.abs(fpr - 0.05).argmin()
threshold = thresholds[idx]

# Print the threshold
print("Threshold for a false positive rate of 5%: {:.4f}".format(threshold))

Threshold for a false positive rate of 5%: 0.6542


# 7.) If the company makes .02*amt on True transactions and loses -amt on False (Use original data)

In [16]:
from sklearn.metrics import confusion_matrix

In [17]:
# Fit the logistic regression model
log_reg_o = LogisticRegression().fit(X, y)

# Make predictions on the original data
y_pred = log_reg_o.predict_proba(X)[:,1]

In [18]:
# Create a dataframe to store the columns
Revenue_df = pd.DataFrame(None)

Revenue_df['amt'] = X['amt']
Revenue_df['is_fraud'] = y
Revenue_df['y_pred'] = y_pred

# Predicted Negative and is True --> 2% on amt
# Predicted Negative and is False --> -100% on amt
Revenue_df['TN'] = np.where((Revenue_df['y_pred'] < threshold) & (Revenue_df['is_fraud'] == 0), 1, 0)
Revenue_df['FN'] = np.where((Revenue_df['y_pred'] < threshold) & (Revenue_df['is_fraud'] == 1), 1, 0)
Revenue_df['revenue'] = 0.02*Revenue_df['TN']*Revenue_df['amt'] - Revenue_df['FN']*Revenue_df['amt']
Revenue_df

,amt,is_fraud,y_pred,TN,FN,revenue
0,2.86,0,0.002945,1,0,0.0572
1,29.84,0,0.003558,1,0,0.5968
2,41.28,0,0.003840,1,0,0.8256
3,60.05,0,0.003486,1,0,1.2010
4,3.19,0,0.003741,1,0,0.0638
...,...,...,...,...,...,...
555714,43.77,0,0.003913,1,0,0.8754
555715,111.84,0,0.004005,1,0,2.2368
555716,86.88,0,0.003865,1,0,1.7376
555717,7.99,0,0.003776,1,0,0.1598


In [19]:
Revenue_df['revenue'].sum()

-400411.7619999998

# 8.) Using Logistic Regression Lasso to inform you. Would you use the selected features in a trusted prediction model?

In [20]:
from sklearn import preprocessing

In [21]:
# Scale the data
scaler = preprocessing.StandardScaler().fit(X)
X_scale = scaler.transform(X)
X_scale

array([[-0.42446311,  0.81652054, -0.49367731, ..., -0.2851168 ,
        -0.31371195, -0.18004671],
       [-0.25233729, -0.2926852 , -0.49367731, ..., -0.2851168 ,
        -0.31371195, -0.18004671],
       [-0.17935288, -0.17885341, -0.49367731, ..., -0.2851168 ,
        -0.31371195, -0.18004671],
       ...,
       [ 0.11156401, -0.28142652, -0.49367731, ..., -0.2851168 ,
        -0.31371195, -0.18004671],
       [-0.39173496, -0.29326111, -0.49367731, ..., -0.2851168 ,
        -0.31371195,  5.55411434],
       [-0.19944911,  0.09247663, -0.49367731, ..., -0.2851168 ,
        -0.31371195, -0.18004671]])

In [26]:
# Fit the model
log_lasso = LogisticRegression(penalty='l1', C=1/0.01, solver='saga', max_iter=10000).fit(X_scale, y)

In [27]:
log_lasso.coef_

array([[ 0.32942184, -0.12325657, -0.02521903, -0.01999391,  0.18411017,
         0.07126578,  0.50464839, -0.01489905, -0.04990822, -0.03573982,
         0.3789472 ,  0.05889515,  0.04620782,  0.4716197 ,  0.17096704,
        -0.87716128]])

In [28]:
# Look at the predictors
varlist = X.columns
varlist

Index(['amt', 'city_pop', 'time_var', 'category_food_dining',
       'category_gas_transport', 'category_grocery_net',
       'category_grocery_pos', 'category_health_fitness', 'category_home',
       'category_kids_pets', 'category_misc_net', 'category_misc_pos',
       'category_personal_care', 'category_shopping_net',
       'category_shopping_pos', 'category_travel'],
      dtype='object')

In [29]:
Coef = pd.DataFrame(None)
Coef['predictor'] = varlist
Coef['coefficient'] = log_lasso.coef_[0]
Coef

,predictor,coefficient
0,amt,0.329422
1,city_pop,-0.123257
2,time_var,-0.025219
3,category_food_dining,-0.019994
4,category_gas_transport,0.184110
5,category_grocery_net,0.071266
6,category_grocery_pos,0.504648
7,category_health_fitness,-0.014899
8,category_home,-0.049908
9,category_kids_pets,-0.035740


The coefficients are all different from 0 in the lasso logistics model. Therefore, I would use the selected features in a trusted prediction model.